In [1]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()

Checking Tensorflow Version:
Tensorflow built with CUDA? True
Tensorflow version: 2.10.0
# GPU available: 1
CUDA: 64_112 | cuDNN: 64_8
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
proxy.load_data()
proxy.process_data()

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
normalized - X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
Train - X: (750, 64, 64, 4) | y: (750, 60, 64, 64, 2)
Test  - X: (250, 64, 64, 4) | y: (250, 60, 64, 64, 2)


In [3]:
model = proxy.make_model()
fit   = proxy.train()

# Parameters: 1,116,630


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/custom_loss/strided_slice_33/StridedSliceGrad' defined at (most recent call last):
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Misael Morales\AppData\Roam

In [ ]:
y_pred = model(proxy.X_train[:3])
print(y_pred.shape)

In [ ]:
fig, axs = plt.subplots(3, 12, figsize=(20,6))
for i in range(3):
    for j in range(12):
        axs[i,j].imshow(y_pred[i,j*5,:,:,-1], 'jet')
        axs[i,j].set(xticks=[], yticks=[])
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 12, figsize=(20,6))
for i in range(3):
    for j in range(12):
        axs[i,j].imshow(proxy.y_train[i,j*5,:,:,-1], 'jet')
        axs[i,j].set(xticks=[], yticks=[])
plt.show()